# 🧮 PyKwant Tutorial: Numerical Methods

This notebook introduces the `pykwant.numerics` module.

In a functional library, numerical algorithms (like interpolation or differentiation) are treated as **Higher-Order Functions**.

* **Input**: Data points or functions.

* **Output**: New functions (Callables) that perform the calculation.

This approach allows us to compose complex models (like a Yield Curve) from simple building blocks without defining heavy classes.

## 1. Setup and Imports

In [1]:
import math
from pykwant import numerics

# Check the module documentation to see available algorithms
print("--- Module Documentation ---")
print(numerics.__doc__)

--- Module Documentation ---

Numerics Module

This module provides fundamental numerical algorithms implemented using a
functional programming paradigm.

It includes functionality for:
- **Interpolation**: Creating interpolation functions (Linear, Log-Linear) as closures.
- **Differentiation**: Computing numerical derivatives via higher-order functions.
- **Root Finding**: Solving equations (e.g., $f(x) = y$) using the Newton-Raphson method.

Unlike traditional vector-oriented scientific libraries (like NumPy), this module
focuses on creating composable *Callables*.



C:\Users\Bonifacio\Documents\Progetti\pykwant\src\pykwant\rates.py:97: SyntaxWarning: "\l" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\l"? A raw string is also an option.
  $DF(t) = e^{-r t}$ implies $r = -\ln(DF) / t$
C:\Users\Bonifacio\Documents\Progetti\pykwant\src\pykwant\rates.py:133: SyntaxWarning: "\l" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\l"? A raw string is also an option.
  $F = \frac{1}{\tau} \left( \frac{DF(T_1)}{DF(T_2)} - 1 \right)$ (Simple Compounding approximation)
C:\Users\Bonifacio\Documents\Progetti\pykwant\src\pykwant\rates.py:166: SyntaxWarning: "\_" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\_"? A raw string is also an option.
  float: The discounted value ($Amount \times DF(payment\_date)$).
C:\Users\Bonifacio\Documents\Progetti\pykwant\src\pykwant\numerics.py:95: SyntaxWarning: "\l" is an invalid escape sequence. Suc

## 2. Linear Interpolation

The `linear_interpolation` function takes lists of x and y coordinates and returns a new function $f(x)$ that estimates values between them.

In [2]:
# 1. Define Data Points
x_data = [1.0, 2.0, 3.0]
y_data = [10.0, 20.0, 30.0]

# 2. Create the Interpolator Function (Closure)
linear_fn = numerics.linear_interpolation(x_data, y_data)

print("--- Linear Interpolation ---")
print(f"Known Point (2.0): {linear_fn(2.0)}")  # Expected: 20.0
print(f"Mid Point (1.5):   {linear_fn(1.5)}")  # Expected: 15.0 (Midpoint)
print(f"Extrapolated (4.0):{linear_fn(4.0)}")  # Expected: 40.0 (Slope continues)

--- Linear Interpolation ---
Known Point (2.0): 20.0
Mid Point (1.5):   15.0
Extrapolated (4.0):40.0


### Handling Extrapolation

Sometimes we want to strictly forbid calculations outside the known range. We can use the `extrapolate=False` flag.

In [3]:
bounded_fn = numerics.linear_interpolation(x_data, y_data, extrapolate=False)

print(f"No Extrap (4.0):   {bounded_fn(4.0)}")  # Expected: nan

No Extrap (4.0):   nan


## 3. Log-Linear Interpolation (Discount Factors)

In Finance, **Discount Factors** ($DF$) decay exponentially with time: $DF(t) = e^{-rt}$.Using simple linear interpolation on DFs is inaccurate. Instead, we use **Log-Linear Interpolation**: we interpolate linearly on $\ln(y)$ and then exponentiate.

In [4]:
print("\n--- Log-Linear vs Linear (Finance Context) ---")

# Setup: 5% continuous rate
r = 0.05
times = [1.0, 2.0]
dfs = [math.exp(-r * 1.0), math.exp(-r * 2.0)]  # DF at year 1 and 2

# Create both interpolators
lin_curve = numerics.linear_interpolation(times, dfs)
log_curve = numerics.log_linear_interpolation(times, dfs)

# Check value at 1.5 years
t_mid = 1.5
exact_df = math.exp(-r * t_mid)

print(f"Exact DF (1.5y):     {exact_df:.6f}")
print(f"Linear Interp:       {lin_curve(t_mid):.6f} (Overestimates)")
print(f"Log-Linear Interp:   {log_curve(t_mid):.6f} (Exact for exponentials)")


--- Log-Linear vs Linear (Finance Context) ---
Exact DF (1.5y):     0.927743
Linear Interp:       0.928033 (Overestimates)
Log-Linear Interp:   0.927743 (Exact for exponentials)


## 4. Numerical Differentiation

The `numerical_derivative` function transforms a function $f(x)$ into its approximate derivative $f'(x)$. This is the engine behind our Risk module (Duration/Convexity).

In [5]:
print("\n--- Numerical Differentiation ---")


# Define a cubic function: f(x) = x^3
def cubic(x: float) -> float:
    return x**3


# Get the derivative function automatically
# Uses central difference method
deriv_fn = numerics.numerical_derivative(cubic, h=1e-5)

# Verify at x=2.0
# Analytical: f'(x) = 3x^2 -> 3 * 2^2 = 12
x_val = 2.0
analytical = 3 * (x_val**2)
numerical = deriv_fn(x_val)

print(f"f(x) = x^3 at x={x_val}")
print(f"Analytical f'(x): {analytical:.6f}")
print(f"Numerical f'(x):  {numerical:.6f}")


--- Numerical Differentiation ---
f(x) = x^3 at x=2.0
Analytical f'(x): 12.000000
Numerical f'(x):  12.000000


## 5. Root Finding (Newton-Raphson)

The `newton_solve` function finds the input $x$ that results in a specific target $y$.

**Financial Use Case**: Calculating the **Implied Rate** (Yield) of a Zero Coupon Bond.

* **Problem**: A Zero Bond pays 100.0 in 2 years. Its current Market Price is 90.0. What is the continuous interest rate $r$?

* **Equation**: $100 \times e^{-r \times 2} = 90$

In [6]:
print("\n--- Root Finding (Implied Rate) ---")

face_value = 100.0
T = 2.0
market_price = 90.0


# Define pricing function P(r)
def price_function(r: float) -> float:
    return face_value * math.exp(-r * T)


# Solve for r such that P(r) == 90.0
implied_r = numerics.newton_solve(
    func=price_function,
    target=market_price,
    guess=0.05,  # Initial guess 5%
    tol=1e-8,
)

print(f"Market Price: {market_price}")
print(f"Implied Rate: {implied_r:.6%}")

# Verification
check_price = price_function(implied_r)
print(f"Check Price:  {check_price:.4f}")


--- Root Finding (Implied Rate) ---
Market Price: 90.0
Implied Rate: 5.268026%
Check Price:  90.0000
